### TODO: realizar generalização na copia do dataset e nao na referencia

In [19]:
import random as rdm
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv("salario.csv")
df.head()

,id,pais,estado,cidade,genero,data,salario
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87


In [21]:
df.describe()

,id
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


## Unindo os atributos de localidade

In [22]:
ids = df.id
cities = df.cidade
states = df.estado
countries = df.pais

In [23]:
locality = []
for i in range(len(cities)):
    locality.append([ids[i], str('{}, {}, {}'.format(cities[i], states[i], countries[i]))])

In [24]:
df_locality = pd.DataFrame(locality, columns=['id', 'localidade'])
df_locality.head()

,id,localidade
0,1,"Mobile, Alabama, United States"
1,2,"Rego, Porto, Portugal"
2,3,"Emmaboda, Kalmar, Sweden"
3,4,"Sala, Västmanland, Sweden"
4,5,"Quimperlé, Bretagne, France"


## Unindo os dataframes

In [25]:
join_dataset = pd.concat([df, df_locality], axis=1, join='inner')
join_dataset.head()

,id,pais,estado,cidade,genero,data,salario,id,localidade
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86,1,"Mobile, Alabama, United States"
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82,2,"Rego, Porto, Portugal"
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85,3,"Emmaboda, Kalmar, Sweden"
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76,4,"Sala, Västmanland, Sweden"
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87,5,"Quimperlé, Bretagne, France"


## Removendo atributos

In [26]:
dataset = join_dataset.drop(columns = ['id', 'pais', 'estado', 'cidade', 'salario'])
dataset.head()

,genero,data,localidade
0,Male,10/01/1987,"Mobile, Alabama, United States"
1,Male,29/11/2007,"Rego, Porto, Portugal"
2,Male,23/06/1984,"Emmaboda, Kalmar, Sweden"
3,Female,04/11/1983,"Sala, Västmanland, Sweden"
4,Female,18/11/1990,"Quimperlé, Bretagne, France"


## Função de generalização da data

In [27]:
def date_generalization(dataset, level):
    if level == 1:
        # date '**/MM/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**' + dataset.iloc[data]['data'][2:10])
    elif level == 2:
        # date '**/**/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**' + dataset.iloc[data]['data'][5:10])
    elif level == 3:
        # date'**/**/****'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**/****')
        
    return dataset

In [28]:
def locality_generalization(dataset, level):
    if level == 0:
        # locality 'city'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = dataset.iloc[data]['localidade'].split(',')[0]
    elif level == 1:
        # locality 'state,'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = dataset.iloc[data]['localidade'].split(',')[1]
    elif level == 2:
        # locality 'country'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = dataset.iloc[data]['localidade'].split(',')[2]
    elif level == 3:
        # locality '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = '*'
        
    return dataset

In [29]:
def gender_generalization(dataset, level):
    if level == 1:
        # gender '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['genero'] = '*'
            
    return dataset

## Gerando as Classes de Equivalência

In [30]:
dataset_test = dataset.head(500)
dataset_test.head()

date_generalization(dataset_test, 2)
dataset_test

,genero,data,localidade
0,Male,**/**/1987,"Mobile, Alabama, United States"
1,Male,**/**/2007,"Rego, Porto, Portugal"
2,Male,**/**/1984,"Emmaboda, Kalmar, Sweden"
3,Female,**/**/1983,"Sala, Västmanland, Sweden"
4,Female,**/**/1990,"Quimperlé, Bretagne, France"
...,...,...,...
495,Female,**/**/2008,"Pemex, Veracruz Llave, Mexico"
496,Male,**/**/1987,"Paradela, Bragança, Portugal"
497,Female,**/**/2000,"Sarzedo, Coimbra, Portugal"
498,Female,**/**/1994,"Newport News, Virginia, United States"


In [31]:
dataset_test.groupby('data')["genero"].count()

data
**/**/1980     6
**/**/1981    26
**/**/1982    11
**/**/1983    13
**/**/1984    22
**/**/1985    18
**/**/1986    16
**/**/1987    13
**/**/1988    19
**/**/1989    14
**/**/1990    20
**/**/1991    10
**/**/1992    10
**/**/1993    17
**/**/1994    25
**/**/1995    23
**/**/1996    19
**/**/1997    14
**/**/1998    14
**/**/1999    22
**/**/2000    24
**/**/2001    15
**/**/2002    13
**/**/2003    16
**/**/2004    21
**/**/2005    19
**/**/2006     9
**/**/2007    15
**/**/2008    17
**/**/2009    11
**/**/2010     8
Name: genero, dtype: int64

## Função de Precisão

(sum i -> Na (sum j -> |D|(h/(HGV Ai))))/ (|D| * Na)

- D: conjunto de dados
- Na: número de atributos semi-identificadores
- h: altura da hierarquia de generalização de valor do atributo Ai após anonimização
- |HGVAi|: altura máxima da hierarquia

In [32]:
def precision(D, Na, h, HGVA):
    summation = 0
    for i in range(Na):
        for j in range (D):
            summation += (h[j] / (HGVA[i]))
    
    return 1 - (summation / (D * Na))
            

In [33]:
def soma(x,y):
    print()

# Gerar dados K-anonimizados

In [34]:
def locality_generalization2(dataset, level):
    if level == 0:
        # locality 'city'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = locais[0]+","+locais[1]+","+locais[2]
    elif level == 1:
        # locality 'state,'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**,"+locais[1]+","+locais[2]
    elif level == 2:
        # locality 'country'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**,**,"+locais[2]
    elif level == 3:
        # locality '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = '**'
        
    return dataset

In [35]:
k = 1 #2 4 8 16
df_faltante = dataset.copy()
d = {'data': [], 'genero': [], 'localidade':[]}
df_final = pd.DataFrame(data=d)

Ordem_Generalizacoes = [(date_generalization,1),(locality_generalization2,1),
                        (date_generalization,2),(locality_generalization2,2),
                        (gender_generalization,1),(date_generalization,3),
                        (locality_generalization2,3),(soma,1)]

for generalizacao_Atula in Ordem_Generalizacoes:
    
    print("------------@@@@@@@@@@@@-----------",end=" => ")
    print(generalizacao_Atula)
    
    aux = df_faltante.groupby(["data","genero","localidade"])["localidade"].count()

    atributos_K_anonimizados = aux[aux >= k].index
    print(atributos_K_anonimizados)

    for i in atributos_K_anonimizados:
        data = i[0]
        genero = i[1]
        localidade = i[2]

        indices_prontos = df_faltante[ (df_faltante['data'] == data) & (df_faltante['genero'] == genero) & (df_faltante["localidade"] == localidade) ].index#[0]
        

        df_final = df_final.append( df_faltante.iloc[indices_prontos], ignore_index = True )
        df_faltante.drop(indices_prontos , inplace=True)
        df_faltante = df_faltante.reset_index(drop=True)#
        
    generalizacao_Atula[0](df_faltante, generalizacao_Atula[1])

print(df_faltante.describe())
print("\n")
print(df_final.describe())

arquivo = open("output.xlsx",'w')
arquivo.write( df_final.to_csv(index = False) )
arquivo.close()

------------@@@@@@@@@@@@----------- => (<function date_generalization at 0x7fde7079b8c8>, 1)
MultiIndex([('**/**/1980', 'Female', ...),
            ('**/**/1980', 'Female', ...),
            ('**/**/1980', 'Female', ...),
            ('**/**/1980',   'Male', ...),
            ('**/**/1980',   'Male', ...),
            ('**/**/1980',   'Male', ...),
            ('**/**/1981', 'Female', ...),
            ('**/**/1981', 'Female', ...),
            ('**/**/1981', 'Female', ...),
            ('**/**/1981', 'Female', ...),
            ...
            ('31/03/1987', 'Female', ...),
            ('31/03/1997', 'Female', ...),
            ('31/05/1984', 'Female', ...),
            ('31/05/2001', 'Female', ...),
            ('31/05/2007', 'Female', ...),
            ('31/08/1987',   'Male', ...),
            ('31/10/1986',   'Male', ...),
            ('31/10/2007',   'Male', ...),
            ('31/12/1989',   'Male', ...),
            ('31/12/2005',   'Male', ...)],
           names=['data', 'gen